# 使用 Dynex 有效探索苯酚衍生物

从大量可能的化合物中进行分子筛选是一项具有挑战性的任务。二次无约束二元优化 (QUBO) 求解器的出现提供了解决此问题的替代方案。我们提出了一种通过集成 QUBO 求解器和密度泛函理论 (DFT) 计算来筛选分子的过程。作为概念验证工作，我们将筛选酚类抑制剂的问题映射到 QUBO 模型上。我们通过借助 DFT 计算修改基团贡献法 (GCM)，将 -OH 键的键解离能 (BDE)（良好聚合物抑制剂的指标）近似引入 QUBO 模型。我们证明了该 QUBO 模型与 DFT 数据之间存在很强的相关性，在 85 个给定分子上进行测试时，相关系数和 Spearman 系数分别为 0.82 和 0.86。这种映射使我们能够通过 QUBO 求解器识别候选者，其 BDE 也通过 DFT 计算进行验证。我们的工作为将 GCM 融入 QUBO 求解器以解决分子筛选问题提供了一个有前途的方向。

https://pubs.acs.org/doi/full/10.1021/acs.iecr.3c03331

In [44]:
from pyqubo import Binary, Constraint, SubH
from pyqubo import UnaryEncInteger,LogEncInteger
import numpy as np
import csv
import time
import dimod

In [3]:
def csv_to_matrix(filename, size):
    num_rows, num_cols = size
    matrix = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            
            if len(row) != num_cols:
                raise ValueError("Number of columns in row does not match expected number of columns")
            matrix_row = []
            for value in row:
                try:
                    matrix_row.append(float(value))
                except ValueError:
                    matrix_row.append(value)
            matrix.append(matrix_row)
    if len(matrix) != num_rows:
        raise ValueError("Number of rows in matrix does not match expected number of rows")
    return matrix

## 我们从csv文件中输入通过DFT计算计算出的权重系数

In [46]:
# 在我们的概念验证工作中，有 5 个位点和 11 种不同的分子结构可供选择。
num_of_position, num_of_mol = 5,11

# 来自一个功能组的贡献。
w = csv_to_matrix('phenol-data/0117data_one_functional group.csv', (5,11))

# 两个官能团相互作用的贡献
W10 = csv_to_matrix('phenol-data/0117data_R1R2_double_functional group.csv', (11,11))
W34 = csv_to_matrix('phenol-data/0117data_R1R2_double_functional group.csv', (11,11))
W21 = csv_to_matrix('phenol-data/0117data_R2R3_double_functional group.csv', (11,11))
W23 = csv_to_matrix('phenol-data/0117data_R2R3_double_functional group.csv', (11,11))

# 第二最近邻交互
W04 = csv_to_matrix('phenol-data/0117data_R1R5_double_functional group.csv', (11,11))
W20 = csv_to_matrix('phenol-data/0117data_R1R3_double_functional group.csv', (11,11))
W24 = csv_to_matrix('phenol-data/0117data_R1R3_double_functional group.csv', (11,11))
W31 = csv_to_matrix('phenol-data/0117data_R2R4_double_functional group.csv', (11,11))
W30 = csv_to_matrix('phenol-data/0117data_R1R4_double_functional group.csv', (11,11))
W14 = csv_to_matrix('phenol-data/0117data_R1R4_double_functional group.csv', (11,11))

### 这里我们将变量标记为$x_{ij}$，其中$i$标记不同的位点，$j$标记不同的功能组。

In [11]:
# 创建二进制变量
def create_x_vars(num_of_position, num_of_mol):
    a = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    for i in range(num_of_position):
        for j in range(num_of_mol):
            vars_name = 'x'+str(i)+'_'+str(j)
            a[i][j] = Binary(vars_name)
    return a

### 我们根据权重系数构建目标函数。

In [12]:
def create_x_vars_name(num_of_position, num_of_mol):
    a0 = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    a1 = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    for i in range(num_of_position):
        for j in range(num_of_mol):
            vars_name = 'x'+str(i)+'_'+str(j)
            a1[i][j] = vars_name
    return a1

## 我们将论文中提到的约束施加到我们的目标函数上。

In [13]:
# 构造目标函数
x = create_x_vars(num_of_position, num_of_mol)
summ = 0
for i in range(num_of_position):
    for j in range(num_of_mol):
        summ += w[i][j]*x[i][j]
        

summ_10 = 0
summ_21 = 0
summ_23 = 0
summ_34 = 0
summ_04 = 0

summ_20 = 0
summ_24 = 0

summ_31 = 0
summ_30 = 0
summ_14 = 0


for i in range(1,num_of_mol):
    for j in range(1,num_of_mol):
        # 最近邻交互
        summ_10 += W10[j][i]*x[1][j]*x[0][i]
        summ_34 += W34[j][i]*x[3][j]*x[4][i] 

        summ_21 += W21[j][i]*x[2][j]*x[1][i]  
        summ_23 += W23[j][i]*x[2][j]*x[3][i]

        summ_04 += W04[j][i]*x[0][j]*x[4][i] 

        # 第二最近邻交互
        summ_20 += W20[j][i]*x[2][j]*x[0][i] 
        summ_24 += W24[j][i]*x[2][j]*x[4][i]

        summ_31 += W31[j][i]*x[3][j]*x[1][i] 
        summ_30 += W30[j][i]*x[3][j]*x[0][i] 
        summ_14 += W14[j][i]*x[1][j]*x[4][i]

obj = (summ_10 + summ_34 + summ_21 + summ_23 + summ_04 + summ_20 + summ_24 + summ_31 + summ_30 + summ_14 ) + summ  + 87.5

In [15]:
H = obj
H_model = H.compile()
H_qubo, offset = H_model.to_qubo()
print(offset)

87.5


## 使用 Dynex 寻找 QUBO 模型的低能耗解决方案。

In [43]:
import dynex
sampleset = dynex.sample_qubo(H_qubo, offset, mainnet=True, num_reads=50000, annealing_time = 2000);
print('Result:')
print(sampleset)

[DYNEX] PRECISION SET TO 0.001
[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬─────────────┬───────────┬───────────────────────────┬─────────┬─────────┬────────────────╮
│   DYNEXJOB │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CHIPS   │ STEPS   │ GROUND STATE   │
├────────────┼─────────────┼───────────┼───────────────────────────┼─────────┼─────────┼────────────────┤
│         -1 │           0 │           │ *** WAITING FOR READS *** │         │         │                │
╰────────────┴─────────────┴───────────┴───────────────────────────┴─────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.00 SECONDS
[DYNEX] SAMPLESET READY
Result:
  x0_1 x0_10 x0_2 x0_3 x0_4 x0_5 x0_6 x0_7 x0_8 ... x4_9     energy num_oc.
0    0     1    1    1    1    1    1    0    0 ...    0 -152.03753       1
['BINARY', 1 rows, 1 samples, 50 variables]


In [41]:
# 从 prometheus_client 导入样本
from dimod.views import samples

optimal = []
for i in samples.sample:
    if samples.sample[i]==1:
        optimal.append(i)

In [42]:
print('The lowest BDE is:',samples.energy+offset, 'kcal/mol')
print('with the structure:', optimal)

The lowest BDE is: -64.53753022400002 kcal/mol
with the structure: ['x0_10', 'x0_2', 'x0_3', 'x0_4', 'x0_5', 'x0_6', 'x1_10', 'x1_2', 'x1_5', 'x1_6', 'x1_7', 'x1_9', 'x2_1', 'x2_10', 'x2_2', 'x2_5', 'x2_7', 'x2_9', 'x3_10', 'x3_2', 'x3_5', 'x3_6', 'x3_7', 'x3_9', 'x4_10', 'x4_2', 'x4_3', 'x4_4', 'x4_5', 'x4_6']
